# Fine-Tune Whisper For ASR with 🤗 Transformers

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0+ Dua020/cv-dataset", "ur/en", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0+Dua020/cv-dataset ", "ur/en", split="test", use_auth_token=True)

print(common_voice)

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

## Prepare Feature Extractor, Tokenizer and Data

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")

### Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-lage-v3", language="ur/en", task="transcribe")

### Combine To Create A WhisperProcessor

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", language="ur/en", task="transcribe")

### Prepare Data

Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.



In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

## Training and Evaluation

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

In [ ]:
model.generation_config.language = "ur/en"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Define the Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-large-v3",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=1000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

In [ ]:
trainer.train()

### Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:


from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test dataset
predictions = trainer.predict(common_voice["test"])

# Get the predicted transcripts
pred_transcripts = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Get the ground truth transcripts
true_transcripts = common_voice["test"]["labels"]



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(true_transcripts, pred_transcripts, average='macro')
recall = recall_score(true_transcripts, pred_transcripts, average='macro')
f1 = f1_score(true_transcripts, pred_transcripts, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


In [ ]:
import jiwer
from jiwer import wer

# Calculate Accuracy
correct = sum(1 for true, pred in zip(true_transcripts, pred_transcripts) if true == pred)
accuracy = correct / len(true_transcripts)

print("Accuracy:", accuracy)

# Calculate Word Error Rate (WER)
wer_score = wer(true_transcripts, pred_transcripts)
print("Word Error Rate (WER):", wer_score)

# Calculate Sentence Error Rate (SER)
ser_score = sum(1 for true, pred in zip(true_transcripts, pred_transcripts) if true != pred) / len(true_transcripts)
print("Sentence Error Rate (SER):", ser_score)

# Calculate Character Error Rate (CER)
cer_score = jiwer.wer(true_transcripts, pred_transcripts, truth_transform=jiwer.remove_punctuation)
print("Character Error Rate (CER):", cer_score)

### Manual Testing

In [ ]:
result = pipe("file31.ogg",generate_kwargs={"language": "en"})  #qammar mehmood
print(result["text"])

In [ ]:
result = pipe("file14.ogg", generate_kwargs={"language": "en"})  #rawalpindi
print(result["text"])

In [ ]:
result = pipe("file17.ogg", generate_kwargs={"language": "en"})  #Punjab college Ayub park campus
print(result["text"])

In [ ]:
result = pipe("file55.ogg", generate_kwargs={"language": "en"})  #Arsal Adnan
print(result["text"])

In [ ]:
result = pipe("audio67.ogg", generate_kwargs={"language": "en"})#Dua Batool
print(result["text"])

In [ ]:
%%time
result = pipe("name1.ogg", generate_kwargs={"language": "en"})#Ghazia Rashid
print(result["text"])


In [ ]:
%%time
result = pipe("name2.ogg", generate_kwargs={"language": "en"})#Miss Asma Sheraz
print(result["text"])

In [ ]:
%%time
result = pipe("file22.ogg", generate_kwargs={"language": "en"})#Muhammad Hasnat Malik
print(result["text"])

In [ ]:
%%time
result = pipe("file76.ogg", generate_kwargs={"language": "en"})#Taimoor Pervaiz
print(result["text"])